In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from SS_tf_var_funcs.ipynp import conv_layer
from SS_tf_var_funcs.ipynp import pooling_layer
from SS_tf_var_funcs.ipynp import fc_layer




(3, 3, 3, 64)


In [ ]:

batch_size = 100
num_of_classes = np.shape(train_Y)[-1]

        
        
def FCN(X):   
    conv_1_1 = conv_layer(X, flt_name = 'conv1_1__W', bias_name = 'conv1_1_b', name = 'conv_1_1')
    conv_1_2 = conv_layer(conv_1_1, flt_name = 'conv1_2_W', bias_name = 'conv1_2_b', name = 'conv_1_2')
    pool_1   = pooling_layer(conv_1_2, name = 'pool_1')

    conv_2_1 = conv_layer(pool_1, flt_name = 'conv2_1_W', bias_name = 'conv2_1_b', name = 'conv_2_1')
    conv_2_2 = conv_layer(conv_2_1, flt_name = 'conv2_2_W', bias_name = 'conv2_2_b', name = 'conv_2_2')
    pool_2   = pooling_layer(conv_2_2, name = 'pool_2')

    conv_3_1 = conv_layer(pool_2, flt_name = 'conv3_1_W', bias_name = 'conv3_1_b', name = 'conv_3_1')
    conv_3_2 = conv_layer(conv_3_1, flt_name = 'conv3_2_W', bias_name = 'conv3_2_b', name = 'conv_3_2')
    conv_3_3 = conv_layer(conv_3_2, flt_name = 'conv3_3_W', bias_name = 'conv3_3_b', name = 'conv_3_3')
    pool_3   = pooling_layer(conv_3_3, name = 'pool_3')

    conv_4_1 = conv_layer(pool_3, flt_name = 'conv4_1_W', bias_name = 'conv4_1_b', name = 'conv_4_1')
    conv_4_2 = conv_layer(conv_4_1, flt_name = 'conv4_2_W', bias_name = 'conv4_2_b', name = 'conv_4_2')
    conv_4_3 = conv_lauyer(conv_4_2, flt_name = 'conv4_3_W', bias_name = 'conv4_3_b', name = 'conv_4_3')
    pool_4   = pooling_layer(conv_4_3, name = 'pool_4')

    conv_5_1 = conv_layer(pool_4, flt_name = 'conv5_1_W', bias_name = 'conv5_1_b', name = 'conv_5_1')
    conv_5_2 = conv_layer(conv_5_1, flt_name = 'conv5_2_W', bias_name = 'conv5_2_b', name = 'conv_5_2')
    conv_5_3 = conv_layer(conv_5_2, flt_name = 'conv5_3_W', bias_name = 'conv5_3_b', name = 'conv_5_3')
    pool_5   = pooling_layer(conv_5_3, name = 'pool_5')

    fc_1     = fc_layer(pool_5, 'fc6_W', 'fc6_b')
    fc_2     = fc_layer(fc_1, 'fc7_W', 'fc7_b')
    fc_3     = fc_layer(fc_2)


    upsample_op_shape    = [batch_size, img_height, img_width, num_of_classes]
    fc3_upsample         = upsample(fc_3, upsample_op_shape, num_of_classes, 'fc3_upsmp', 32)

    pool4_skip_flt_shape = [1, 1, np.shape(pool_4)[-1], num_of_classes]
    pool3_skip_flt_shape = [1, 1, np.shape(pool_3)[-1], num_of_classes]

    pool4_skip           = skip_connection(pool_4, 'pool4_skip', pool4_skip_flt_shape)
    pool4_upsample       = upsample(pool4_skip, upsample_op_shape, num_of_classes, 'pool4_upsmp', 16)

    pool3_skip           = skip_connection(pool_3, 'pool3_skip', pool3_skip_flt_shape)
    pool3_upsample       = upsample(pool3_skip, upsample_op_shape, num_of_classes, 'pool3_upsmp', 8)

    logits = tf.add(fc3_upsample, tf.add(2 * pool4_upsample, 4 * pool3_upsample))

    return logits
    
        

def train_optimizer(x_batch, y_batch):
    FCN_logits = FCN(x_batch)
    cost = logistic_loss(logits=FCN_logits, labels=y_batch, n_classes=num_of_classes)
    optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)
    
    return FCN_logits, cost, optimizer


def run_FCN_model():
    with tf.Session() as session:
        x_ph, y_ph, dropout   = get_placeholders_for_train_label_data()
        FCN_logits, loss, optimizer = train_optimizer(x_ph, y_ph)
        
        session.run(tf.global_variables_initializer())
        saver = tf.train.saver(tf.trainable_variables())
        
        for epoch in range(0, 1):
            for i in range(0, len(train_X), batch_size):
                train_x = train_X[i : i + batch_size]
                train_y = train_Y[i : i + batch_size]
                _, loss = session.run([optimizer, loss], feed_dict = {x_ph : train_x,
                                                                      y_ph : train_y})
                
        
        save_path = saver.save(session, "./SS_FCN_model")
        
    return FCN_logits
